In [ ]:
!pip install langchain
!pip install openai
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.6 MB/s eta 0:00:00


In [ ]:
import os
import openai

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 객체를 생성합니다.
chat_prompt_template = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
chat_model = ChatOpenAI()
output_parser = StrOutputParser() # Message의 content를 추출해서 string으로 변환

# 체인을 정의합니다. (호출은 아직 하지 않았습니다.)
chain = chat_prompt_template | chat_model | output_parser

In [ ]:
# invoke 인자로 dict를 넘깁니다.
# dict는 key : value 구성되어 있습니다. "사과" : "과일 중 하나이며..."
# 그 이유가 prompt_template이 dict를 받기 때문입니다.

chain.invoke({"topic": "ice cream"})

'Why did the ice cream truck break down? Because it had too many "scoops"!'

In [ ]:
# 객체를 따로 생성하지 않고, 체인 정의할 때 바로 생성자를 이용

chain = chat_prompt_template | ChatOpenAI() | StrOutputParser()
chain.invoke({"topic": "ice cream"})

'Why did the ice cream go to therapy? Because it was feeling a little Rocky Road!'

### dict말고 "ice cream"만 넣고 싶을 때

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

chat_prompt_template = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
chat_model = ChatOpenAI()
output_parser = StrOutputParser()

chain = {"topic": RunnablePassthrough()} | chat_prompt_template | chat_model | output_parser

In [ ]:
# invoke할 때, 굳이 topic 키로 dict로 넣어야 하나? 그냥 문자열로 넣으면 안될까?
# 문자열을 받아서, chain에 전달됨
# prompt_template의 입력 형식 dict로 받아야한다.
# 인자를 value로 dict를 구성해서 넘거야겠다.
# "ice cream" > RunnablePassthrough() > {"topic": RunnablePassthrough()} > prompt_template

chain.invoke("ice cream")

# RunnablePassthrough() = "ice cream"

# 질문 : RunnablePassthrough() 형식이 문자열인가요?

'Why did the ice cream truck break down? Because it had too many "scoops"!'

In [ ]:
# 답 : invoke의 형식을 그대로 따릅니다. 만약 문자열이면 문자열, dict이면 dict입니다.

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

prompt_template = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
chat_model = ChatOpenAI()
output_parser = StrOutputParser()

# 이렇게도 가능합니다.
chain = RunnablePassthrough() | prompt_template | chat_model | output_parser

chain.invoke({"topic": "ice cream"})

'Why did the ice cream truck break down?\nBecause it had too many "scoops"!'

```
# object approach
chain = a.__or__(b)
chain("some input")

# pipe approach
chain = a | b
chain("some input")

```



In [ ]:
class Runnable:

    def __init__(self, func):
        self.func = func

    # 이 메서드는 파이썬의 비트 OR 연산자(|)를 오버로드합니다.
    def __or__(self, other):

        def chained_func(*args, **kwargs):
            # the other func consumes the result of this func
            return other(self.func(*args, **kwargs))

        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [ ]:
def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

In [ ]:
# wrap the functions with Runnable
add_five = Runnable(add_five)
#multiply_by_two = Runnable(multiply_by_two)

In [ ]:
# chain the runnable functions together
chain = add_five | multiply_by_two

In [ ]:
# invoke the chain
chain(3)  # we should return ??

16

In [ ]:
# run them using the object approach
chain = add_five.__or__(multiply_by_two)
chain(3)  # should return 16

16

In [ ]:
def num2ko(x):
    return str(x) + "입니다."

In [ ]:
chain = add_five | multiply_by_two | num2ko

In [ ]:
chain(3)

# num2ko는 Runnable이 아닌데, 호출이 되나요?

'16입니다.'

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# 템플릿 만드는 방법

# 미션 : 어떤 주제에 대해서 사용자 레벨에 맞게 선택된 언어로 대화 예시를 만드는 앱을 개발!
# 미션 : [어떤] 주제에 대해서 사용자 [레벨]에 맞게 [선택된 언어]로 대화 예시를 만드는 앱을 개발!

# 기획
# "Please generate dialogue sentences in English on the topic of health for a beginner level."
# "Please generate dialogue sentences in Korean on the topic of AI for an intermediate level."

# 양식과 변수를 분리하기
# "Please generate three sentences in a dialogue in (English) on the topic of (health) for a (beginner) level."
# "Please generate three sentences in a dialogue in (Korean) on the topic of (AI) for an (intermediate) level."

# "Please generate three sentences in a dialogue in {language} on the topic of {topic} for a/an {level} level."

# 양식 조정하기
# "Please generate three sentences in a dialogue in {language} on the topic of {topic} for a level of {level}."

In [ ]:
template = "Please generate three sentences in a dialogue in {language} on the topic of {topic} for a level of {level}."

chat_prompt_template = ChatPromptTemplate.from_template(template)

chain = chat_prompt_template | chat_model | StrOutputParser()

# 변수는 dict로 넣습니다.
output = chain.invoke({"language" : "English", "topic" : "travel", "level" : "beginner"})

print(output)

Person 1: Where do you want to go on your next trip?
Person 2: I want to visit Paris, it's my dream destination.
Person 1: That sounds amazing, I hope you have a great time exploring the city of lights.


In [ ]:
# 변수 중에 시스템에서 입력해야할 것과 사용자가 입력해야할 것 분리
# language : 설정
# topic : 바뀐다.
# level : 설정

def get_learning_language(_):
    print("###")
    print(_)
    print("in get_learning_language")
    print("###")
    return "English"

def get_learning_level(_):
    print("###")
    print(_)
    print("in get_learning_level")
    print("###")
    return "beginner"

# 그럼 이 함수를 체인 내에서 어떻게 호출해야하는가?
# 예시 ) 휴가 신청서 사용할 때, 신청일시, 이름 등이 자동으로 기입되는 것

In [ ]:
# 방법 1

template = "Please generate three sentences in a dialogue in {language} on the topic of {topic} for a level of {level}."

prompt_template = ChatPromptTemplate.from_template(template)

chain = prompt_template | chat_model| StrOutputParser()

# invoke 하기 전에 필요한 정보를 dict로 구성하는 방법
output = chain.invoke({"language" : get_learning_language(''), "topic" : "travel", "level" : get_learning_level('')})

print(output)

###

in get_learning_language
###
###

in get_learning_level
###
Person 1: Have you ever traveled to another country before?
Person 2: Yes, I went to Spain last year. It was amazing!
Person 1: I hope to travel abroad someday too. Where would you recommend I visit?


In [ ]:
# 방법2

from langchain_core.runnables import RunnablePassthrough

template = "Please generate three sentences in a dialogue in {language} on the topic of {topic} for a level of {level}."

chat_prompt_template = ChatPromptTemplate.from_template(template)

# 함수의 인자는 무조건 1개이어야 합니다.
chain = (
    RunnablePassthrough.assign(language = get_learning_language,
                               level = get_learning_level)

    | chat_prompt_template
    | chat_model
    | StrOutputParser()
)

# {"topic" : "travel"} > dict language, level 추가 > {"topic", ... "language", ... "level" ...} dict가 완성
# > prompt_template > chat_model

output = chain.invoke({"topic" : "travel"}) # 변수 1개

print(output)

# 넘어오는 인자에 추가적으로 key와 value를 넣고 싶을 때, RunnablePassthrough을 사용한다.

######
{'topic': 'travel'}
in get_learning_level
###

{'topic': 'travel'}
in get_learning_language
###
Person A: "Have you ever been on a plane before?"
Person B: "No, I haven't. I'm excited to go on my first flight next month."
Person A: "That's cool! Where are you going?"


In [ ]:
chain

RunnableAssign(mapper={
  language: RunnableLambda(get_learning_language),
  level: RunnableLambda(get_learning_level)
})
| ChatPromptTemplate(input_variables=['language', 'level', 'topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'level', 'topic'], template='Please generate three sentences in a dialogue in {language} on the topic of {topic} for a level of {level}.'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x798d78d8f070>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x798d78bf2920>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnablePassthrough

template = "Please generate three sentences in a dialogue in {language} on the topic of {topic} for a level of {level}."

prompt_template = ChatPromptTemplate.from_template(template)

chain = (
    {"topic" : RunnablePassthrough()}
    | RunnablePassthrough.assign(language = get_learning_language,
                                 level = get_learning_level)

    | prompt_template
    | chat_model
    | StrOutputParser()
)

print(chain)

output = chain.invoke("travel")

print(output)

first={
  topic: RunnablePassthrough()
} middle=[RunnableAssign(mapper={
  language: RunnableLambda(get_learning_language),
  level: RunnableLambda(get_learning_level)
}), ChatPromptTemplate(input_variables=['language', 'level', 'topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'level', 'topic'], template='Please generate three sentences in a dialogue in {language} on the topic of {topic} for a level of {level}.'))]), ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x798d78d8f070>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x798d78bf2920>, openai_api_key=SecretStr('**********'), openai_proxy='')] last=StrOutputParser()
###
{'topic': 'travel'}
in get_learning_language
###
###
{'topic': 'travel'}
in get_learning_level
###
Person 1: Have you ever been on a plane before?
Person 2: No, I haven't. I usually just drive to go on vacation.
Person 1: You should try flying sometime, it's so